In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

import os
import numpy as np
import random
import gc

from google.colab import drive
drive.mount('/content/drive')
print(os.listdir("/content/drive/My Drive/CoLab"))
basedir = "/content/drive/My Drive/CoLab"

In [0]:
model  = tf.keras.models.load_model(os.path.join(basedir, "test6"))

In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (2, 2), activation='relu', input_shape=(22, 22, 1)),  
  tf.keras.layers.Conv2D(16, (8, 8), activation='relu', input_shape=(22, 22, 1)),
  tf.keras.layers.Conv2D(32, (2, 2), activation='relu', input_shape=(22, 22, 1)),
  tf.keras.layers.Conv2D(64, (2, 2), activation='relu', input_shape=(22, 22, 1)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(196, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(4, activation='softmax')
])
model.summary()
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])

In [0]:
# TEST 4 DATALOAD

dataset = {}

print("load ")
tmp = np.load(os.path.join(basedir, "X.npy"))
print("X")
tmp2 = np.load(os.path.join(basedir, "Xtest.npy"))
print("Xtest")
dataset = {
    "X" : tmp.reshape(len(tmp), 22, 22, 1),
    "Y" : np.load(os.path.join(basedir, "Y.npy")),
    "Xtest" : tmp2.reshape(len(tmp2), 22, 22, 1),
    "Ytest" : np.load(os.path.join(basedir, "Ytest.npy")),
}
print("Finished")


In [0]:
s = np.arange(dataset["X"].shape[0])
np.random.shuffle(s)

dataset["X"] = dataset["X"][s]
dataset["Y"] = dataset["Y"][s]
print ("y: ", dataset["X"][0])

In [0]:
predY = np.array(model.predict(dataset["Xtest"]))
count = 0
for i in range(0,len(predY)):
  count += 1 if np.argmax(predY[i]) == np.argmax(dataset["Ytest"][i]) else 0

print(count, "/" , len(predY), " : ", count / len(predY) * 100)


In [0]:
  # TEST 4 LEARNING
import datetime
now = datetime.datetime.now
while True:
  history = model.fit(dataset["X"], dataset["Y"], epochs=5)
  with open(os.path.join(basedir,"Log.txt"), 'a') as f:
    f.write(now().strftime('%d %H:%M:%S') + "\t: " + str(model.evaluate(dataset["Xtest"], dataset["Ytest"])) + "\n")
  
  model.save(os.path.join(basedir, "test6"))

In [0]:
  # TEST 3 LEARNING
import datetime
now = datetime.datetime.now
while True:
  gc.collect()
  X = []
  Y = []
  for j in range(50000):
    index = random.randint(5000, len(dataset[3]["X"]) - 1)
    X.append(dataset[3]["X"][index])
    Y.append(dataset[3]["Y"][index])
    index = random.randint(0, len(dataset[4]["X"]) - 1)
    X.append(dataset[4]["X"][index])
    Y.append(dataset[4]["Y"][index])
  X = np.array(X)
  Y = np.array(Y)
  history = model.fit(X, Y, epochs=1)
  del(X)
  del(Y)
  with open(os.path.join(basedir,"Log.txt"), 'a') as f:
    f.write(now().strftime('%d %H:%M:%S') +str(i) + "\t: " + str(history.history) + "\n")
  model.save(os.path.join(basedir, "test4"))

In [0]:
import math
result = {}
a = model.predict(dataset[3]["X"])
y = dataset[3]["Y"]

for i in range(len(a)):
  if np.argmax(a[i]) != np.argmax(y[i]):
    key = math.floor(i/1000)
    if not key in result:
      result[key] = 0
    result[key] += 1
from matplotlib import pyplot as plt
x = []
y = []
for k in result:
  x.append(k)
  y.append(result[k])
plt.plot(x,y)
print(np.sum(y))
del(a)
del(y)
del(x)
del(result)

In [0]:
# TEST2 DATALOAD
sample = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,51,52,53]

dataset = {}

for i in sample:
  print("load " + str(i))
  dataset[i] = {
      "X" : np.load(os.path.join(basedir, "puzzle-in-" + str(i) +".npy")).reshape(100000, 22, 22, 1),
      "Y" : np.load(os.path.join(basedir, "puzzle-out-" + str(i) +".npy")),
  }

In [0]:
while True:
  gc.collect()
  samples = random.sample(sample, 5)
  print(samples)
  X = []
  Y = []
  for i in samples:
    d = dataset[i]
    for dX in d["X"]:
      X.append(dX)
    
    for dY in d["Y"]:
      Y.append(dY)
  X = np.array(X)
  Y = np.array(Y)
  history = model.fit(X, Y, epochs=3)
  del(X)
  del(Y)
  model.fit(dataset[1]["X"], dataset[1]["Y"], epochs=1)
  with open(os.path.join(basedir,"Log.txt"), 'a') as f:
    f.write(str(samples) + " : " + str(history.history) + "\n")
  model.save(os.path.join(basedir, "test2"))

In [0]:
while True:
  for i in sample:
    d = dataset[i]
    print(i)
    history = model.fit(d["X"], d["Y"], epochs=3)

    model.fit(dataset[1]["X"], dataset[1]["Y"], epochs=1)

    with open(os.path.join(basedir,"Log.txt"), 'a') as f:
      f.write(str(i) + "\t: " + str(history.history) + "\n")
    model.save(os.path.join(basedir, "test2"))

In [0]:
str(history.history)

In [0]:
X1 = np.load(os.path.join(basedir, "puzzle-in-1.npy"))
Y1 = np.load(os.path.join(basedir, "puzzle-out-1.npy"))

sample = [2,3,4,5,6,7,8,9,10,11,12,13,14,15]

while True:
  samples = [] #random.sample(sample, 6)
  #samples.append(51)
  #samples.append(52)
  #samples.append(53)
  X = X1.reshape((100000, 22, 22, 1))
  Y = np.copy(Y1)
  for i in samples:
    print(i)
    Xtmp = np.load(os.path.join(basedir, "puzzle-in-"+str(i)+".npy"))
    Ytmp = np.load(os.path.join(basedir, "puzzle-out-"+str(i)+".npy"))
    X = np.concatenate((X, Xtmp.reshape(100000, 22, 22, 1)), axis=0)
    Y = np.concatenate((Y, Ytmp), axis=0)

  model.fit(X, Y, epochs=10)
  a = model.predict(X)
  counter = 0
  for i in range(len(X)):
    pred = np.argmax(a[i])
    ans = np.argmax(Y[i])
    if pred == ans:
      counter += 1
  with open(os.path.join(basedir,"Log.txt"), 'a') as f:
    f.write(str(samples) + " : " + str(counter) + "/" + str(len(X)) + "\n")
  model.save(os.path.join(basedir, "test2"))




In [0]:
loaded = tf.keras.models.load_model(os.path.join(basedir, "test"))
X = np.load(os.path.join(basedir, "puzzle-in-1.npy"))
Y = np.load(os.path.join(basedir, "puzzle-out-1.npy"))
X = X.reshape((100000, 22, 22, 1))


a = loaded.predict(X)

counter = 0
for i in range(100000):
  pred = np.argmax(a[i])
  ans = np.argmax(Y[i])
  if pred == ans:
    counter += 1
print(counter)